In [38]:
import TrainModels 
import ReplaysParser 
import numpy as np

In [ ]:
#lstmWeightsPath = "../TrainedModels/lstm_various-match-length_30000.weights"
lstmWeightsPath = "../TrainedModels/lstm_various-match-length_not-normalized_30000.weights" #best model so far
#lstmWeightsPath = "../TrainedModels/lstm_various-match-length_not-normalized_loss-stop_30000.weights"
#lstmWeightsPath = "../TrainedModels/lstm_various-match-length_not-normalized_loss-stop_10.weights" #benchmark model, basically no training it random guesses at best
lstm = TrainModels.UseTrainedLSTMModel(lstmWeightsPath)

In [ ]:
hits = np.zeros(26)
confidences = np.zeros(26)
matches = np.zeros(26)

matchesCount = 10

for j in range(matchesCount):
    matchLength = len(vectorizedReplayData[0])
    vectorizedReplayData = ReplaysParser.LoadVectorizedData(1, 30150 + i)
    normalizedData, expectedResults =  TrainModels.NormalizeInputAsSequences(vectorizedReplayData, 1, matchLength, 324)
    
    for percent in range(26):
        i = round((percent/26)*matchLength)
        if(percent == 25):
            i = matchLength - 1
        input = np.zeros([1,i+1,324])
        input[0] = normalizedData[0][:i+1]
        expectedOutput = expectedResults[0][0]

        predictions = lstm.predict(input)
        finalPredictions = predictions.argmax(axis=2)[0][-1]
        confidence = max(predictions[0][-1][0],predictions[0][-1][1])

        matches[percent] = matches[percent] + 1
        confidences[percent] = confidences[percent] + confidence
        if(finalPredictions == expectedOutput):
            hits[percent] = hits[percent] + 1

    print("Analyzed matches "+str(j)+"/"+str(matchesCount),end='\r')

print (hits/matchesCount)
print (confidences/matchesCount)